In [1]:
%pip install matplotlib pandas numpy seaborn catboost xfeat

Note: you may need to restart the kernel to use updated packages.


In [2]:
# ========================================
# Library
# ========================================
import math
import random
import pickle
import itertools
import warnings
warnings.filterwarnings('ignore')
import time
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
# import jpholiday
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import (
    TimeSeriesSplit,
    StratifiedKFold,
    KFold,
    GroupKFold,
    StratifiedGroupKFold,
)
from xfeat import SelectCategorical, LabelEncoder, LambdaEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer 
# ←←←いくつかいらないものがありそう。xfeatの使い方をおさらいしないと
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from scipy.optimize import minimize
import lightgbm as lgb
import optuna
# catBoost
from catboost import CatBoost, CatBoostClassifier, Pool
# XGBoost

from expressway_router import ExpresswayRouter

from collections import Counter

In [3]:
i_path = '../../train/'
o_path = '../model/'

TARGET = ['is_congestion']

threshold = 0.5

## Load datas

In [4]:
train_df = pd.read_csv(i_path + 'train.csv')
road_df = pd.read_csv(i_path + 'road_local.csv')
search_spec_df = pd.read_csv(i_path + 'search_specified.csv')
search_unspec_df = pd.read_csv(i_path + 'search_unspecified.csv')

## Preprocessing

In [5]:
def expand_datetime(df):
    if 'datetime' in df.columns:
        df['year'] = df['datetime'].dt.year
        df['month'] = df['datetime'].dt.month
        df['day'] = df['datetime'].dt.day
        df['hour'] = df['datetime'].dt.hour
    if 'date' in df.columns:
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
    return df

In [6]:
def preprocess_fit(train_df, search_spec_df, search_unspec_df):
    train_df['datetime'] = pd.to_datetime(train_df['datetime'])
    search_spec_df['datetime'] = pd.to_datetime(search_spec_df['datetime'])
    search_unspec_df['date'] = pd.to_datetime(search_unspec_df['date'])

    train_df = expand_datetime(train_df)
    search_unspec_df = expand_datetime(search_unspec_df)

    train_df = train_df.merge(search_spec_df, on=['datetime', 'start_code', 'end_code'], how='left')
    train_df = train_df.merge(search_unspec_df, on=['year', 'month', 'day', 'start_code', 'end_code'], how='left')
    train_df = train_df.merge(road_df.drop(['start_name', 'end_name'], axis=1), on=['start_code', 'end_code'], how='left')

    train_df['dayofweek'] = train_df['datetime'].dt.weekday

    train_df['section'] = train_df['start_code'].astype(str) + '_' + train_df['KP'].astype(str) + '_' + train_df['end_code'].astype(str)

    return train_df

In [7]:
train = preprocess_fit(train_df, search_spec_df, search_unspec_df)

In [8]:
train.head(5)

,datetime,start_code,end_code,KP,OCC,allCars,speed,is_congestion,year,month,day,hour,search_specified,date,search_unspecified,road_code,direction,limit_speed,start_KP,end_KP,start_pref_code,end_pref_code,start_lat,end_lat,start_lng,end_lng,start_degree,end_degree,dayofweek,section
0,2021-04-08 00:00:00,1110210,1800006,2.48,1.833333,507,94.208661,0,2021,4,8,0,15.0,2021-04-08,3419.0,1800,下り,100.0,0.8,9.4,13,11,35.75582,35.80615,139.601514,139.535511,4.0,2.0,3,1110210_2.48_1800006
1,2021-04-08 01:00:00,1110210,1800006,2.48,1.750000,444,94.469663,0,2021,4,8,1,6.0,2021-04-08,3419.0,1800,下り,100.0,0.8,9.4,13,11,35.75582,35.80615,139.601514,139.535511,4.0,2.0,3,1110210_2.48_1800006
2,2021-04-08 02:00:00,1110210,1800006,2.48,1.500000,363,92.593407,0,2021,4,8,2,3.0,2021-04-08,3419.0,1800,下り,100.0,0.8,9.4,13,11,35.75582,35.80615,139.601514,139.535511,4.0,2.0,3,1110210_2.48_1800006
3,2021-04-08 03:00:00,1110210,1800006,2.48,1.583333,430,94.501160,0,2021,4,8,3,26.0,2021-04-08,3419.0,1800,下り,100.0,0.8,9.4,13,11,35.75582,35.80615,139.601514,139.535511,4.0,2.0,3,1110210_2.48_1800006
4,2021-04-08 04:00:00,1110210,1800006,2.48,1.750000,500,94.079840,0,2021,4,8,4,30.0,2021-04-08,3419.0,1800,下り,100.0,0.8,9.4,13,11,35.75582,35.80615,139.601514,139.535511,4.0,2.0,3,1110210_2.48_1800006


In [9]:
#########################################################
# ここからtrainのデータの整形などを実施する(ドラプラの元データとかは使わんでもある程度出来そうな気がしてる。。。。。。。。)
#########################################################


def preprocess(df):
    agg_dfs = []
    num_df = SelectNumerical().fit_transform(df)
    encoder = Pipeline([
        SelectCategorical(),
        LabelEncoder(output_suffix=""),
        ])
    le_df = encoder.fit_transform(df)
    # agg_dfs.append(get_agg_df()) # group byすることによるそれぞれの
    preprocessed_df = pd.concat([num_df,le_df]+agg_dfs, axis=1)
    return preprocessed_df

In [10]:
cat_cols = ['road_code', 'start_code', 'end_code', 'section', 'direction', 'dayofweek'] # この箇所については、select_numericalあたりを使ったほうが良さそう

num_cols = ['year', 'month', 'day', 'hour', 'search_specified', 'search_unspecified', 'KP', 'start_KP', 'end_KP', 'limit_speed', 'OCC']
feature_cols = cat_cols + num_cols

In [11]:
test = train.loc[(train['year']==2021) & ((train['month']==4) | (train['month']==5))]
train = train.loc[~((train['year']==2021) & ((train['month']==4) | (train['month']==5)))]

In [12]:
train[feature_cols].head(5)

,road_code,start_code,end_code,section,direction,dayofweek,year,month,day,hour,search_specified,search_unspecified,KP,start_KP,end_KP,limit_speed,OCC
1296,1800,1110210,1800006,1110210_2.48_1800006,下り,1,2021,6,1,0,12.0,2656.0,2.48,0.8,9.4,100.0,2.083333
1297,1800,1110210,1800006,1110210_2.48_1800006,下り,1,2021,6,1,1,3.0,2656.0,2.48,0.8,9.4,100.0,1.750000
1298,1800,1110210,1800006,1110210_2.48_1800006,下り,1,2021,6,1,2,11.0,2656.0,2.48,0.8,9.4,100.0,1.250000
1299,1800,1110210,1800006,1110210_2.48_1800006,下り,1,2021,6,1,3,28.0,2656.0,2.48,0.8,9.4,100.0,1.916667
1300,1800,1110210,1800006,1110210_2.48_1800006,下り,1,2021,6,1,4,15.0,2656.0,2.48,0.8,9.4,100.0,2.000000


In [13]:
train.columns

Index(['datetime', 'start_code', 'end_code', 'KP', 'OCC', 'allCars', 'speed',
       'is_congestion', 'year', 'month', 'day', 'hour', 'search_specified',
       'date', 'search_unspecified', 'road_code', 'direction', 'limit_speed',
       'start_KP', 'end_KP', 'start_pref_code', 'end_pref_code', 'start_lat',
       'end_lat', 'start_lng', 'end_lng', 'start_degree', 'end_degree',
       'dayofweek', 'section'],
      dtype='object')

In [14]:
le_dict = {}
for c in tqdm(cat_cols):
    le = LabelEncoder()
    train[c] = le.fit_transform(train[c])
    le_dict[c] = le

100%|██████████| 6/6 [00:01<00:00,  3.82it/s]


## Training

In [15]:
def train_lgbm(X, y, cv, model_path=[], params={}, verbose=100):

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records), dtype=np.float32)

    def objective(trial):
        lgb_params = {
            "objective": "binary",
            "metric": "binary_logloss",
            "boosting_type": "rf",
            "verbosity": -1,
            "boost_from_average": False,
            "random_seed": 42,
            "feature_pre_filter": False,
            "max_depth": trial.suggest_int('max_depth', 4, 8),
            "num_leaves": trial.suggest_int("num_leaves", 2, 100),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.1, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.1, 1.0),
            "min_child_samples": trial.suggest_int("min_child_samples", 1, 25),
            "min_data_in_leaf": trial.suggest_int('min_data_in_leaf', 1, 4),
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
            "min_split_gain": trial.suggest_float("min_split_gain", 1e-4, 1e-1, log=True),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
            "bagging_seed": 42,
            "verbosity": -1,
        }


        model = lgb.LGBMClassifier(**lgb_params)

        model.fit(x_train, y_train,
                  eval_set=[(x_valid, y_valid)],
                  callbacks=[
                      lgb.early_stopping(stopping_rounds=50, verbose=True),
                      lgb.log_evaluation(100)
                  ]
                  )

        pred_y = model.predict_proba(x_valid)[:, 1]
        auc = roc_auc_score(y_valid, pred_y)

        return auc

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=10)

        best_params = study.best_params
        clf = lgb.LGBMClassifier(**best_params)

        clf.fit(x_train, y_train,
                eval_set=[(x_valid, y_valid)],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=50, verbose=True),
                    lgb.log_evaluation(100)
                ]
                )

        pred_i = clf.predict_proba(x_valid)[:, 1]
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        score = roc_auc_score(y_valid, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = roc_auc_score(y, oof_pred)

    print("=" * 50)
    print(f"FINISH: CV Score: {score:.4f}")
    return score, oof_pred, models

In [16]:
N_SPLIT = 5
kf = StratifiedGroupKFold(N_SPLIT)
cv_list = list(kf.split(train, y=train[TARGET], groups=train['date']))

X = train[feature_cols].values
y = train[TARGET].values

print('train shape:', train.shape)

# training
lgb_score, lgb_oof_pred, lgb_models = train_lgbm(X, y=y, cv=cv_list)

train shape: (5429424, 30)


[I 2024-03-20 05:39:05,112] A new study created in memory with name: no-name-73424d81-db60-4e18-86a0-11d84e1b2ed1


Training until validation scores don't improve for 50 rounds


[I 2024-03-20 05:39:08,994] Trial 0 finished with value: 0.9850188152480994 and parameters: {'max_depth': 5, 'num_leaves': 96, 'learning_rate': 0.04600508397026657, 'feature_fraction': 0.26600824398443706, 'bagging_fraction': 0.1511654842770972, 'min_child_samples': 25, 'min_data_in_leaf': 4, 'lambda_l1': 0.012025760088628401, 'lambda_l2': 4.286588091207017e-05, 'min_split_gain': 0.0006508859042694555, 'bagging_freq': 2}. Best is trial 0 with value: 0.9850188152480994.


Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.128938
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[26]	valid_0's binary_logloss: 0.133176


...

Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.128837
Training until validation scores don't improve for 50 rounds


[I 2024-03-20 05:43:32,019] Trial 8 finished with value: 0.9765446971736508 and parameters: {'max_depth': 7, 'num_leaves': 65, 'learning_rate': 0.08788056211108848, 'feature_fraction': 0.7264600910430782, 'bagging_fraction': 0.5759790229277664, 'min_child_samples': 22, 'min_data_in_leaf': 4, 'lambda_l1': 0.00047537800634342577, 'lambda_l2': 1.5467102660566567e-08, 'min_split_gain': 0.0019344255461316556, 'bagging_freq': 4}. Best is trial 3 with value: 0.9861656319271503.


Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.12882
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[11]	valid_0's binary_logloss: 0.128814


[I 2024-03-20 05:43:36,998] Trial 9 finished with value: 0.9710756124562612 and parameters: {'max_depth': 7, 'num_leaves': 51, 'learning_rate': 0.03535917566943197, 'feature_fraction': 0.9561999016312624, 'bagging_fraction': 0.7272600792478007, 'min_child_samples': 13, 'min_data_in_leaf': 3, 'lambda_l1': 0.0005875263401978008, 'lambda_l2': 5.4149808710628045e-05, 'min_split_gain': 0.0018382408211251647, 'bagging_freq': 4}. Best is trial 3 with value: 0.9861656319271503.


[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=8 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] feature_fraction is set=0.9392666622259336, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9392666622259336
[LightGBM] [Warning] lambda_l2 is set=0.0002767524467166091, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0002767524467166091
[LightGBM] [Warning] lambda_l1 is set=1.8059402002920946e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8059402002920946e-07
[LightGBM] [Warning] bagging_fraction is set=0.25173416333674, subsample=1.0 will be ignored. Current value: bagging_fraction=0.25173416333674
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=8 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] feature_fraction is set=0.9392666622259336, colsample_b

In [17]:
def train_catboost(X, y, cv, model_path=[], params={}, verbose=100):

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records), dtype=np.float32)

    def objective(trial):
        cb_params = {
            "iterations": trial.suggest_int('iterations', 2, 6),
            "objective": "Logloss",
            "eval_metric": "Logloss",
            "boosting_type": "Plain",
            "boost_from_average": False,
            "random_seed": 42,
            "max_depth": trial.suggest_int('max_depth', 4, 8),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 100.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.1, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.1, 1.0),
            "grow_policy": "Lossguide",
        }


        model = CatBoost(cb_params)

        model.fit(c_train)

        pred_y = model.predict(c_valid)
        auc = roc_auc_score(y_valid, pred_y)

        return auc

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]
        c_train = Pool(x_train, label=y_train)
        c_valid = Pool(x_valid, label=y_valid)
        
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=10)

        best_params = study.best_params
        model = CatBoostClassifier(**best_params)

        model.fit(c_train)

        c
        oof_pred[idx_valid] = pred_y
        models.append(model)
        score = roc_auc_score(y_valid, pred_y)
        print(f" - fold{i + 1} - {score:.4f}")

    score = roc_auc_score(y, oof_pred)

    print("=" * 50)
    print(f"FINISH: CV Score: {score:.4f}")
    return score, oof_pred, models

In [18]:
cb_score, cb_oof_pred, cb_models = train_catboost(X, y=y, cv=cv_list)

[I 2024-03-20 05:44:02,791] A new study created in memory with name: no-name-6eb59d0d-2ab8-4776-a0df-4229d99931d8


0:	learn: 0.6580338	total: 179ms	remaining: 358ms
1:	learn: 0.6250497	total: 296ms	remaining: 148ms
2:	learn: 0.5946475	total: 330ms	remaining: 0us


[I 2024-03-20 05:44:03,932] Trial 0 finished with value: 0.966357770072902 and parameters: {'iterations': 3, 'max_depth': 6, 'learning_rate': 0.03622715393576466, 'reg_lambda': 4.845627015197551e-06, 'subsample': 0.569992941751225, 'colsample_bylevel': 0.13615863016378052}. Best is trial 0 with value: 0.966357770072902.


0:	learn: 0.6880220	total: 168ms	remaining: 335ms
1:	learn: 0.6829514	total: 282ms	remaining: 141ms


[I 2024-03-20 05:44:04,978] Trial 1 finished with value: 0.9694402942134253 and parameters: {'iterations': 3, 'max_depth': 6, 'learning_rate': 0.005145151282299943, 'reg_lambda': 4.573850429723252e-08, 'subsample': 0.1537637127281278, 'colsample_bylevel': 0.7988603655483268}. Best is trial 1 with value: 0.9694402942134253.


...

0:	learn: 0.6496362	total: 159ms	remaining: 477ms
1:	learn: 0.6098922	total: 317ms	remaining: 317ms
2:	learn: 0.5733836	total: 469ms	remaining: 156ms
3:	learn: 0.5397599	total: 629ms	remaining: 0us


[I 2024-03-20 05:46:15,258] Trial 8 finished with value: 0.9840798459672528 and parameters: {'iterations': 4, 'max_depth': 4, 'learning_rate': 0.04462557089973648, 'reg_lambda': 2.4018678641558386, 'subsample': 0.6625401258635375, 'colsample_bylevel': 0.31805089467726344}. Best is trial 8 with value: 0.9840798459672528.


0:	learn: 0.6460733	total: 142ms	remaining: 283ms
1:	learn: 0.6033387	total: 232ms	remaining: 116ms


[I 2024-03-20 05:46:16,125] Trial 9 finished with value: 0.9569003367178752 and parameters: {'iterations': 3, 'max_depth': 8, 'learning_rate': 0.04843779190463181, 'reg_lambda': 2.7239091253696996e-07, 'subsample': 0.4085133143667451, 'colsample_bylevel': 0.4247063307396602}. Best is trial 8 with value: 0.9840798459672528.


2:	learn: 0.5643861	total: 432ms	remaining: 0us
0:	learn: 0.6496495	total: 107ms	remaining: 320ms
1:	learn: 0.6103103	total: 214ms	remaining: 214ms
2:	learn: 0.5742214	total: 316ms	remaining: 105ms
3:	learn: 0.5405578	total: 417ms	remaining: 0us
 - fold5 - 0.5000
FINISH: CV Score: 0.8417


In [19]:
def train_xgb(X, y, cv, model_path=[], params={}, verbose=100):

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records), dtype=np.float32)

    def objective(trial):
        xgb_params = {
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "booster": "gbtree",
            # "verbosity": 0,
            "n_jobs": -1,
            "seed": 42,
            "max_depth": trial.suggest_int('max_depth', 4, 8),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-5, 1e2, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
            "gamma": trial.suggest_float("gamma", 0, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)
        evals = [(dtrain, 'train'), (dvalid, 'eval')]

        model = xgb.train(xgb_params, dtrain, num_boost_round=1000,
                          early_stopping_rounds=50, evals=evals, verbose_eval=verbose)

        pred_y = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
        auc = roc_auc_score(y_valid, pred_y)

        return auc

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_train, y_train = X[idx_train], y[idx_train]
        X_valid, y_valid = X[idx_valid], y[idx_valid]

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=10)

        best_params = study.best_params
        xgb_params = {
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "booster": "gbtree",
            "verbosity": 0,
            "n_jobs": -1,
            "seed": 42,
            **best_params,
        }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)
        evals = [(dtrain, 'train'), (dvalid, 'eval')]

        model = xgb.train(xgb_params, dtrain, num_boost_round=1000,
                          early_stopping_rounds=50, evals=evals, verbose_eval=verbose)

        pred_i = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = roc_auc_score(y_valid, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = roc_auc_score(y, oof_pred)

    print("=" * 50)
    print(f"FINISH: CV Score: {score:.4f}")
    return score, oof_pred, models

In [ ]:
xgb_score, xgb_oof_pred, xgb_models = train_catboost(X, y=y, cv=cv_list)

[I 2024-03-20 05:46:36,058] A new study created in memory with name: no-name-04b2299f-f668-4dfe-9d1f-6e0a466e59a3


0:	learn: 0.6267388	total: 266ms	remaining: 266ms
1:	learn: 0.5689424	total: 453ms	remaining: 0us


[I 2024-03-20 05:46:37,128] Trial 0 finished with value: 0.952396034815814 and parameters: {'iterations': 2, 'max_depth': 8, 'learning_rate': 0.0689476623087299, 'reg_lambda': 4.4583556630536574e-07, 'subsample': 0.7826883563194972, 'colsample_bylevel': 0.5831409400263383}. Best is trial 0 with value: 0.952396034815814.


0:	learn: 0.6631919	total: 109ms	remaining: 544ms
1:	learn: 0.6347265	total: 192ms	remaining: 384ms
2:	learn: 0.6079236	total: 363ms	remaining: 363ms
3:	learn: 0.5826613	total: 436ms	remaining: 218ms
4:	learn: 0.5589862	total: 577ms	remaining: 115ms
5:	learn: 0.5364986	total: 727ms	remaining: 0us


[I 2024-03-20 05:46:38,424] Trial 1 finished with value: 0.9716386532688309 and parameters: {'iterations': 6, 'max_depth': 5, 'learning_rate': 0.030817002919564015, 'reg_lambda': 5.812843005717101e-05, 'subsample': 0.6480239388396605, 'colsample_bylevel': 0.18634694566298424}. Best is trial 1 with value: 0.9716386532688309.


...

4:	learn: 0.4631398	total: 571ms	remaining: 0us
0:	learn: 0.6201890	total: 256ms	remaining: 512ms
1:	learn: 0.5581349	total: 307ms	remaining: 154ms


[I 2024-03-20 05:48:23,076] Trial 8 finished with value: 0.9688356587953594 and parameters: {'iterations': 3, 'max_depth': 6, 'learning_rate': 0.07626032702640256, 'reg_lambda': 0.2520039638502834, 'subsample': 0.9297500746503258, 'colsample_bylevel': 0.35662692286875786}. Best is trial 3 with value: 0.9845227367298944.


2:	learn: 0.5035675	total: 492ms	remaining: 0us
0:	learn: 0.6364073	total: 68.3ms	remaining: 205ms
1:	learn: 0.5860043	total: 164ms	remaining: 164ms
2:	learn: 0.5409173	total: 229ms	remaining: 76.3ms


[I 2024-03-20 05:48:23,881] Trial 9 finished with value: 0.9719500166544734 and parameters: {'iterations': 4, 'max_depth': 7, 'learning_rate': 0.05856117429652094, 'reg_lambda': 1.1145647496070223e-07, 'subsample': 0.21054847148689843, 'colsample_bylevel': 0.8593886025980819}. Best is trial 3 with value: 0.9845227367298944.


3:	learn: 0.5003302	total: 357ms	remaining: 0us
0:	learn: 0.6819949	total: 164ms	remaining: 656ms
1:	learn: 0.6710895	total: 324ms	remaining: 487ms
2:	learn: 0.6604270	total: 494ms	remaining: 330ms
3:	learn: 0.6499961	total: 653ms	remaining: 163ms
4:	learn: 0.6397957	total: 822ms	remaining: 0us
 - fold4 - 0.8913


In [ ]:
# 最適な閾値を探索
def tune_threshold(train, oof_pred):
    """
    閾値の探索
    """
    def func(x_list, df, oof):
        score = f1_score(df[TARGET], oof>x_list[0])
        return -score
    x0 = [0.5]
    result = minimize(func, x0,  args=(train, oof_pred), method="nelder-mead")
    threshold = result.x[0]
    return threshold

In [ ]:
lgb_threshold = tune_threshold(train, lgb_oof_pred)
cb_threshold = tune_threshold(train, cb_oof_pred)
xgb_threshold = tune_threshold(train, xgb_oof_pred)
print(len(lgb_threshold), len(cb_threshold), len(xgb_threshold))

In [ ]:
# testデータのpredict
def test_predict(model, threshold, test=test, how='lgb'):
    """modelのpredictを行ってくれる関数"""
    if how='lgb':
        dataset = Pool(test.drop(columns=TARGET), label=test[TARGET])
    elif how='cb':
        x_train, y_train = X[idx_train], y[idx_train]
        c_train = Pool(x_train, label=y_train)
        pred_y = model.predict(c_valid)
    elif how='xgb':
        dvalid = xgb.DMatrix(X_valid, label=y_valid)
        pred_y = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
        auc = roc_auc_score(y_valid, pred_y)
    probabilities = model.predict_proba(test)[:, 1]
    return (probabilities > threshold).astype(int)

In [ ]:
lgb_predict = test_predict(lgb_model[0], lgb_threshold, test=test, how='lgb')

In [ ]:
lgb_models[0].predict(Pool(test.drop(columns=TARGET), label=test[TARGET]))

In [ ]:
test.drop(columns=TARGET)

In [ ]:
all_predictions = [lgb_predict, cb_predict, xgb_predict]
combined_predictions = np.vstack(all_predictions)
final_predictions = [Counter(row).most_common(1)[0][0] for row in combined_predictions]

In [ ]:
f1_score()

In [ ]:
# with open('../model/model.pickle', mode='wb') as f:
#     pickle.dump(models,f,protocol=2)

In [ ]:
# # ========================================
# # feature importance
# # ========================================
# def visualize_importance(models, feat_train_df):
#     feature_importance_df = pd.DataFrame()
#     for i, model in enumerate(models):
#         _df = pd.DataFrame()
#         _df["feature_importance"] = model.feature_importances_
#         _df["column"] = feat_train_df.columns
#         _df["fold"] = i + 1
#         feature_importance_df = pd.concat([feature_importance_df, _df],
#                                           axis=0, ignore_index=True)

#     order = feature_importance_df.groupby("column")\
#         .sum()[["feature_importance"]]\
#         .sort_values("feature_importance", ascending=False).index

#     fig, ax = plt.subplots(figsize=(12, max(6, len(order) * .25)))
#     sns.boxplot(data=feature_importance_df,
#                   x="feature_importance",
#                   y="column",
#                   order=order,
#                   ax=ax,
#                   palette="viridis",
#                   orient="h")
#     ax.tick_params(axis="x", rotation=90)
#     ax.set_title("Importance")
#     ax.grid()
#     fig.tight_layout()
#     return fig, ax, feature_importance_df

# fig, ax, feature_importance_df = visualize_importance(models, train[feature_cols])

In [ ]:
models